In [1]:
from trees_emulator.load_data import *
import numpy as np
import pickle

In [2]:
# Load Data
data = LoadData(2016, site="MHD", verbose=True)

Loading Meteorology data from data/met/EUROPE*10magl*2016*.nc
Loading footprint data from data/fps/MHD-10magl*2016*.nc
Cutting data to size
Loading extra meteorology from data/met/EUROPE*verticalgradients*2016*.nc and extracting gradients
Extracting wind vectors
All data loaded


In [4]:
# Load trained model. Info is a dict with general info about the training, clfs is a list of length size**2 with all regressors
with open('trained_models/MHD.txt', 'rb') as f:
    [info, clfs] = pickle.load(f)
print("info:", info)
print("first regressor:", clfs[0])

info: {'site': 'MHD', 'training data': '201[4-5]', 'sampling frequency': 2, 'size': 10}
first regressor: GradientBoostingRegressor(loss='absolute_error', max_depth=50,
                          max_features='sqrt', n_estimators=150)


In [4]:
# Get the inputs set up
## variables that are passed at the time of the footprint and x hours before
vars_with_past = [data.y_wind, data.x_wind, data.met.PBLH.values]
hours_before = 6
## variables that are only passed at the time of the footprint
vars_without_past = [data.temp_grad, data.x_wind_grad, data.y_wind_grad]

inputs = get_all_inputs(vars_with_past, 6, vars_without_past)